In [17]:
import ccxt
import numpy as np
import pandas as pd
import talib
from datetime import datetime

In [22]:
# first we need to have a list of ohlcv data
# let's get 1000 bars of 1 hour data from 2024-01-01 00:00:00 UTC

exchange = ccxt.binance()

symbol = 'BTC/USDT'
timeframe = '1h'
limit = 1000
since = exchange.parse8601('2024-01-01T00:00:00Z')

ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
# print(df)

In [23]:
# Create numpy arrays of prices from the ohlcv data
# 0 : timestamp   1 : open   2 : high   3 : low   4 : close   5 : volume

Open = [candle[1] for candle in ohlcv]
Open = np.array(Open)

High = [candle[2] for candle in ohlcv]
High = np.array(High)

Low = [candle[3] for candle in ohlcv]
Low = np.array(Low)

Close = [candle[4] for candle in ohlcv]
Close = np.array(Close)

In [24]:
# Some MAs

sma_20 = talib.SMA(Close, timeperiod=20)
sma_50 = talib.SMA(Close, timeperiod=50)
sma_200 = talib.SMA(Close, timeperiod=200)

ema_20 = talib.EMA(Close, timeperiod=20)
ema_50 = talib.EMA(Close, timeperiod=50)
ema_200 = talib.EMA(Close, timeperiod=200)

wma_20 = talib.WMA(Close, timeperiod=20)
wma_50 = talib.WMA(Close, timeperiod=50)
wma_200 = talib.WMA(Close, timeperiod=200)

trima_20 = talib.TRIMA(Close, timeperiod=20)
trima_50 = talib.TRIMA(Close, timeperiod=50)
trima_200 = talib.TRIMA(Close, timeperiod=200)